# TEAcademy NFT Data 

by: 🐙 Octopus

May 21, 2024

## Objectives

1. Generate a `.csv` file that has all wallet addresses, with boolean information on all TEAcademy NFTs, to be used in simulation. 
2. Improve my own understanding of GraphQL. 

## Initial GraphQL Requests

I used code by Skrillah to request the overall NFT data for the 

In [1]:
import requests
import csv

import pandas as pd 

In [2]:
# Code from Skrillah

def query_subgraph(raft_id):
    """Query the Otterspace subgraph to get specific NFT data by raft ID."""
    query = f"""
    {{
        raft(id:"{raft_id}"){{
            id,
            specs{{
                id
                metadata {{
                    name
                    description
                }}
                totalBadgesCount
                badges{{
                    owner {{
                        id
                    }}
                }}
            }}
        }}
    }}
    """
    url = 'https://api.thegraph.com/subgraphs/name/otterspace-xyz/badges-optimism'
    response = requests.post(url, json={'query': query})
    return response.json()

In [3]:
# From Skrillah
def clean_text(text):
    """Clean text for CSV output."""
    replacements = {
        '\u2013': '-',  # EN Dash
        '\u2014': '-',  # EM Dash
        '\u2028': '',   # Line Separator
        '\u2019': "'",  # Right single quotation mark
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

## Processing Proof of Knowledge NFTs

We begin by uing the dictionary retrieved using Skrillah's `query_subgraph` function to get the data for Proof-of-Knowledge NFTs, corresponding to `rafts:29`. 

In [4]:
proof_of_knowledge_request = query_subgraph("rafts:29")

After a bit of exploring on the structure of the dictionary, I realized that it was a highly nested structure of dictionaries and lists. The following code extracts the names of the NFTs, and gives a list of all owner IDs for each NFT. The final `name`-`owners` correspondence is stored in a dictionary called `proof_of_knowledge_nft_owner_dict`. 

In [5]:
proof_of_knowledge_nft_owner_dict = {}

initial_data = proof_of_knowledge_request['data']['raft']['specs']

for k in range(len(initial_data)):
    specific_entry = initial_data[k]
    name = specific_entry['metadata']['name']
    print(name)
    badges_info = specific_entry['badges']
    # badge_name = badges_info['metadata']['name']
    # print(badge_name)
    badge_owners = [badges_info[p]['owner']['id'] for p in range(len(badges_info))]
    print(badge_owners)
    proof_of_knowledge_nft_owner_dict[name] = badge_owners


TE FUNDAMENTALS – MODULE 4 of 5 – VERSION 2 – 2023
['0xbb8743ea733155fe5e81ed285aea72cc19b2ca87', '0x049debbad61a20e21e872b06ef4f25be1253c802', '0xa481db1ac55683dfb2847c02e417b7e411bcbbea', '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370', '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242', '0xfb2069bf55eb844694d8028df74dafe550821ffc', '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f', '0x5658f4b362e95c0bc32d5ce0051457882db92ee5', '0x6697aeedc3eadbcfb25962c2045aedf438b4c1f2', '0x6a7becf86b56fb68c948cb0a9fe9961370079190', '0xc8eba9130f40a0ea9be530407e5b4f7199118864', '0x6e3556f2c8f29559ed67642a1813b177f1871613', '0x278a4bd5a8d09bcf11b1052ec3129a7efa75d213', '0x28ddb1f987f1d433eb82eef644b006741977f456', '0xfcbc07905fee2d64025461b8ddb27f77f256827f', '0xa587b5aa3ddf49559cde74140456bf65d716f30a', '0x6120faa647cd26cd6b2c1df880eff537065c6815', '0x6d465d2081b799770d0ce7e755d8db1665903ffb', '0x3b067af83f540cb827825a6ee5480441a4237e77', '0x6a5cce4ac8b280ba789eb2e2bd3895b46133e5d4', '0x58e39293ec6ce3f4d49c9a44b

We do some reality testing to make sure that the data structure behaves as expected. 

In [6]:
proof_of_knowledge_nft_owner_dict

{'TE FUNDAMENTALS – MODULE 4 of 5 – VERSION 2 – 2023': ['0xbb8743ea733155fe5e81ed285aea72cc19b2ca87',
  '0x049debbad61a20e21e872b06ef4f25be1253c802',
  '0xa481db1ac55683dfb2847c02e417b7e411bcbbea',
  '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370',
  '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242',
  '0xfb2069bf55eb844694d8028df74dafe550821ffc',
  '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f',
  '0x5658f4b362e95c0bc32d5ce0051457882db92ee5',
  '0x6697aeedc3eadbcfb25962c2045aedf438b4c1f2',
  '0x6a7becf86b56fb68c948cb0a9fe9961370079190',
  '0xc8eba9130f40a0ea9be530407e5b4f7199118864',
  '0x6e3556f2c8f29559ed67642a1813b177f1871613',
  '0x278a4bd5a8d09bcf11b1052ec3129a7efa75d213',
  '0x28ddb1f987f1d433eb82eef644b006741977f456',
  '0xfcbc07905fee2d64025461b8ddb27f77f256827f',
  '0xa587b5aa3ddf49559cde74140456bf65d716f30a',
  '0x6120faa647cd26cd6b2c1df880eff537065c6815',
  '0x6d465d2081b799770d0ce7e755d8db1665903ffb',
  '0x3b067af83f540cb827825a6ee5480441a4237e77',
  '0x6a5cce4ac8b280ba789eb2e2bd389

Here are all the proof-of-knowledge NFTs. There are definitely duplicates, which we will get to later. 

In [7]:
all_proof_of_knowledge_nft_names = list(proof_of_knowledge_nft_owner_dict.keys())
all_proof_of_knowledge_nft_names

['TE FUNDAMENTALS – MODULE 4 of 5 – VERSION 2 – 2023',
 'TE FUNDAMENTALS – MODULE 5 of 5 – VERSION 1 – 2022',
 'TE FUNDAMENTALS – MODULE 3 of 5 – VERSION 1 – 2022',
 'TE FUNDAMENTALS – MODULE 2 of 5 – VERSION 2 – 2023',
 'NFT-based Reputation in Web3 – V1 – 2023',
 'TE FUNDAMENTALS – MODULE 3 of 5 – VERSION 2 – 2023',
 'TE FUNDAMENTALS – MODULE 5 of 5 – VERSION 2 – 2023',
 'TE FUNDAMENTALS – MODULE 2 of 5 – VERSION 1 – 2022',
 'TE FUNDAMENTALS – MODULE 4 of 5 – VERSION 1 – 2022 ',
 'TE FUNDAMENTALS – MODULE 1 of 5 – VERSION 2 – 2023',
 'TE FUNDAMENTALS – MODULE 1 of 5 – VERSION 1 – 2022']

In [8]:
len(all_proof_of_knowledge_nft_names)

11

## Repeating the Process for Proof-of-Special-Contribution NFTs

We repeat the process for proof-of-special-contribution NFTS (`rafts:74`), producing another dictionary of NFTs and their corresponding owners. 

In [9]:
proof_of_special_request = query_subgraph("rafts:74")

In [10]:
proof_of_special_nft_owner_dict = {}

initial_data = proof_of_special_request['data']['raft']['specs']

for k in range(len(initial_data)):
    specific_entry = initial_data[k]
    name = specific_entry['metadata']['name']
    print(name)
    badges_info = specific_entry['badges']
    # badge_name = badges_info['metadata']['name']
    # print(badge_name)
    badge_owners = [badges_info[p]['owner']['id'] for p in range(len(badges_info))]
    print(badge_owners)
    proof_of_special_nft_owner_dict[name] = badge_owners

Token Engineering @EthCC Paris 2023 - Speaker
['0xcc449df434d886576202eef7c9ef2aa0948c310b', '0xfcbc07905fee2d64025461b8ddb27f77f256827f', '0xc710f3da5ea5ae7b060166942185458c60c0434f', '0xf3e0a7eb1e8fefe4d77ae7e65153a4b9ab7524fc', '0x4914faf80b6593a8eac644ced7911a6dc9105355', '0x5705b28907558887b023b7dae2a2fe8c28aa91f7', '0xebefe02040c7284a30ade6e52da7e24af6884ffc']
TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023
['0x44a60575bafadbf1471d628e7a8bdad64e2e7d5f']
Token Engineering @EthCC Paris 2023
['0xa4d338186d93a77b5a82ac8253b79050b0058275', '0x272b0f68c9f518d6dc8bd41240506ca1e7f46c57', '0xcca02de5372cb0caac425543487789eef5d17fb5', '0x58c58d02323af6659e8d94fe3feb14385503a8f9', '0x737f8c9b7c07d13c84275ee0890b592160d1f8a3', '0x5658f4b362e95c0bc32d5ce0051457882db92ee5', '0x8bb2884ee943111aa5119a36b30085a15a66c972', '0x9b826b0c6a6601b34f2b9f86f4bcff715e68b2eb', '0xcf79c7eaec5bdc1a9e32d099c5d6bdf67e4cf6e8', '0x11435318c6231f730bb04c651522294021365cfc', '0x444a6e71d2f0fcb4013f987221bed7be0f6d1e

In [11]:
print(proof_of_special_nft_owner_dict)

{'Token Engineering @EthCC Paris 2023 - Speaker': ['0xcc449df434d886576202eef7c9ef2aa0948c310b', '0xfcbc07905fee2d64025461b8ddb27f77f256827f', '0xc710f3da5ea5ae7b060166942185458c60c0434f', '0xf3e0a7eb1e8fefe4d77ae7e65153a4b9ab7524fc', '0x4914faf80b6593a8eac644ced7911a6dc9105355', '0x5705b28907558887b023b7dae2a2fe8c28aa91f7', '0xebefe02040c7284a30ade6e52da7e24af6884ffc'], 'TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023': ['0x44a60575bafadbf1471d628e7a8bdad64e2e7d5f'], 'Token Engineering @EthCC Paris 2023': ['0xa4d338186d93a77b5a82ac8253b79050b0058275', '0x272b0f68c9f518d6dc8bd41240506ca1e7f46c57', '0xcca02de5372cb0caac425543487789eef5d17fb5', '0x58c58d02323af6659e8d94fe3feb14385503a8f9', '0x737f8c9b7c07d13c84275ee0890b592160d1f8a3', '0x5658f4b362e95c0bc32d5ce0051457882db92ee5', '0x8bb2884ee943111aa5119a36b30085a15a66c972', '0x9b826b0c6a6601b34f2b9f86f4bcff715e68b2eb', '0xcf79c7eaec5bdc1a9e32d099c5d6bdf67e4cf6e8', '0x11435318c6231f730bb04c651522294021365cfc', '0x444a6e71d2f0fcb4013f987221

In [12]:
all_proof_of_special_nft_names = list(proof_of_special_nft_owner_dict.keys())
all_proof_of_special_nft_names

['Token Engineering @EthCC Paris 2023 - Speaker',
 'TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023',
 'Token Engineering @EthCC Paris 2023',
 'TE FUNDAMENTALS COURSE AUTHOR – LAUNCH 2022',
 'TE360 STUDY GROUP HOST 2022',
 'TE FUNDAMENTALS STUDY GROUP HOST 2022/2023',
 'Token Engineering Barcamp - Speaker - Paris 2023\u2028',
 'Token Engineering Barcamp - Paris 2023\u2028',
 'Token Engineering Barcamp - Team - Paris 2023\u2028']

## Combining the Information

Now we combine the two dictionaries, to produce a list of all relevant TEAcademy NFTs and their owners. 

In [13]:
combined_nft_owner_dict = {**proof_of_knowledge_nft_owner_dict,
                      **proof_of_special_nft_owner_dict}

all_nft_owner_dict = {clean_text(key): value 
                    for key, value in 
                    combined_nft_owner_dict.items()}

In [14]:
all_nfts = [clean_text(name) 
         for name 
         in list(all_nft_owner_dict.keys())]
all_nfts

['TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 1 - 2022',
 'TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 1 - 2022',
 'TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 2 - 2023',
 'NFT-based Reputation in Web3 - V1 - 2023',
 'TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 1 - 2022',
 'TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 1 - 2022 ',
 'TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 1 - 2022',
 'Token Engineering @EthCC Paris 2023 - Speaker',
 'TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023',
 'Token Engineering @EthCC Paris 2023',
 'TE FUNDAMENTALS COURSE AUTHOR - LAUNCH 2022',
 'TE360 STUDY GROUP HOST 2022',
 'TE FUNDAMENTALS STUDY GROUP HOST 2022/2023',
 'Token Engineering Barcamp - Speaker - Paris 2023',
 'Token Engineering Barcamp - Paris 2023',
 'Token Engineering Barcamp - Team - Pari

Now I want to create a "reversed structure" that I feel will be better suited for conversion to a `pandas DataFrame`, where the key-value pairs are:
* key: `ID`, value: a string giving the owner ID
* For each NFT list, key: `NFT_name`, value: `True` or `False` depending on whether this address owns this NFT. 

The logic will be:
1. Create a combined list of all owner IDs for all NFTs.
2. Looping through each owner ID, create a boolean list of whether the owner has the ID, by checking whether the ID is in that NFT's list of owners. 

In [15]:
all_owners = []

for _, owners_list in all_nft_owner_dict.items():
    new_owners = [owner 
                  for owner 
                  in owners_list 
                  if not (owner in all_owners) ]
    all_owners.extend(new_owners)

Now we double-check that things look reasonable. 

Here are the first addresses who own TEA NFTs. 

In [16]:
all_owners[0:5]

['0xbb8743ea733155fe5e81ed285aea72cc19b2ca87',
 '0x049debbad61a20e21e872b06ef4f25be1253c802',
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea',
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370',
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242']

In [17]:
print(f"There are {len(all_owners)} total IDs who hold TE Academy NFTs.")

There are 347 total IDs who hold TE Academy NFTs.


In [18]:
final_owners_dict = {}

for owner in all_owners:
    new_list = [owner] 
    new_list += [int(owner 
                  in all_nft_owner_dict.get(nft))
                  for nft in all_nfts]
    assert (len(new_list) == (len(all_nfts) + 1)) # Check right length.
    final_owners_dict[owner] = new_list

final_owners_dict


{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87': ['0xbb8743ea733155fe5e81ed285aea72cc19b2ca87',
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0x049debbad61a20e21e872b06ef4f25be1253c802': ['0x049debbad61a20e21e872b06ef4f25be1253c802',
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0],
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea': ['0xa481db1ac55683dfb2847c02e417b7e411bcbbea',
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': ['0x024bd3d1c3b6ba2277a744d7a99fbade9404a370',
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': ['0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242',
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0xfb2069bf55eb844694d80

In [19]:
columns_to_use = ['ID'] + all_nfts
columns_to_use

['ID',
 'TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 1 - 2022',
 'TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 1 - 2022',
 'TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 2 - 2023',
 'NFT-based Reputation in Web3 - V1 - 2023',
 'TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 1 - 2022',
 'TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 1 - 2022 ',
 'TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 2 - 2023',
 'TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 1 - 2022',
 'Token Engineering @EthCC Paris 2023 - Speaker',
 'TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023',
 'Token Engineering @EthCC Paris 2023',
 'TE FUNDAMENTALS COURSE AUTHOR - LAUNCH 2022',
 'TE360 STUDY GROUP HOST 2022',
 'TE FUNDAMENTALS STUDY GROUP HOST 2022/2023',
 'Token Engineering Barcamp - Speaker - Paris 2023',
 'Token Engineering Barcamp - Paris 2023',
 'Token Engineering Barcamp - Team

In [20]:
# Convert dictionary to DataFrame
df = pd.DataFrame.from_dict(final_owners_dict, 
                            orient='index',
                            columns = columns_to_use)

df.reset_index(drop = True, inplace = True)
df.head()

,ID,TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 2 - 2023,TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 1 - 2022,TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 1 - 2022,TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 2 - 2023,NFT-based Reputation in Web3 - V1 - 2023,TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 2 - 2023,TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 2 - 2023,TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 1 - 2022,TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 1 - 2022,...,TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 1 - 2022,Token Engineering @EthCC Paris 2023 - Speaker,TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023,Token Engineering @EthCC Paris 2023,TE FUNDAMENTALS COURSE AUTHOR - LAUNCH 2022,TE360 STUDY GROUP HOST 2022,TE FUNDAMENTALS STUDY GROUP HOST 2022/2023,Token Engineering Barcamp - Speaker - Paris 2023,Token Engineering Barcamp - Paris 2023,Token Engineering Barcamp - Team - Paris 2023
0,0xbb8743ea733155fe5e81ed285aea72cc19b2ca87,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0x049debbad61a20e21e872b06ef4f25be1253c802,1,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0xa481db1ac55683dfb2847c02e417b7e411bcbbea,1,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0x024bd3d1c3b6ba2277a744d7a99fbade9404a370,1,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242,1,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
first_filename = "raw_nft_data_may_22.csv"
df.to_csv(first_filename)

## Final Cleanup

Now we want to do the final cleanup to create two `.csv` files. 

This includes:
- Coming up with reasonable column names.
- Combining duplicate columns. 

In [22]:
# List of keys
keys_list = [
    'ID',
    'TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 2 - 2023',
    'TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 1 - 2022',
    'TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 1 - 2022',
    'TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 2 - 2023',
    'NFT-based Reputation in Web3 - V1 - 2023',
    'TE FUNDAMENTALS - MODULE 3 of 5 - VERSION 2 - 2023',
    'TE FUNDAMENTALS - MODULE 5 of 5 - VERSION 2 - 2023',
    'TE FUNDAMENTALS - MODULE 2 of 5 - VERSION 1 - 2022',
    'TE FUNDAMENTALS - MODULE 4 of 5 - VERSION 1 - 2022 ',
    'TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 2 - 2023',
    'TE FUNDAMENTALS - MODULE 1 of 5 - VERSION 1 - 2022',
    'Token Engineering @EthCC Paris 2023 - Speaker',
    'TE FUNDAMENTALS STUDY GROUP HOST C2 2022/2023',
    'Token Engineering @EthCC Paris 2023',
    'TE FUNDAMENTALS COURSE AUTHOR - LAUNCH 2022',
    'TE360 STUDY GROUP HOST 2022',
    'TE FUNDAMENTALS STUDY GROUP HOST 2022/2023',
    'Token Engineering Barcamp - Speaker - Paris 2023',
    'Token Engineering Barcamp - Paris 2023',
    'Token Engineering Barcamp - Team - Paris 2023'
]

# List of values corresponding to the keys
values_list = [
    'ID',
    'FUND_MOD4_V2',
    'FUND_MOD5_V1',
    'FUND_MOD3_V1',
    'FUND_MOD2_V2',
    'NFTREP_V1',
    'FUND_MOD3_V2',
    'FUND_MOD5_V2',
    'FUND_MOD2_V1',
    'FUND_MOD4_V1',
    'FUND_MOD1_V2',
    'FUND_MOD1_V1',
    'SPEAKER_ETHCC_PARIS23',
    'STUDY_GROUP_HOST_C2_22_23',
    'ETHCC_23',
    'FUND_AUTHOR',
    'STUDY_GROUP_HOST_360_22',
    'STUDY_GROUP_HOST_FUND_22_23',
    'SPEAKER_BARCAMP_PARIS_23',
    'BARCAMP_PARIS_23',
    'TEAM_BARCAMP_PARIS_23'
]

# Create dictionary from the lists
col_replacement_names = dict(zip(keys_list, values_list))


In [23]:
# Create a new DataFrame with renamed columns
new_col_names_df = df.rename(columns=col_replacement_names)

In [24]:
second_filename = "nft_data_may_22_new_column_names.csv"
new_col_names_df.to_csv(second_filename)

Now we process the data to combine certain pieces of information:
* Only one value for each TE Fundamentals Module, taken as the maximum of the two versions. 
* Only one value for Study Group Hosts, taken as the maximum of the three columns. 

In [25]:
combined_df = new_col_names_df.copy()

In [26]:
# Take the pairwise maximum of the columns
combined_df["FUND_MOD1"] = combined_df[["FUND_MOD1_V1", "FUND_MOD1_V2"]].max(axis=1)
combined_df["FUND_MOD2"] = combined_df[["FUND_MOD2_V1", "FUND_MOD2_V2"]].max(axis=1)
combined_df["FUND_MOD3"] = combined_df[["FUND_MOD3_V1", "FUND_MOD3_V2"]].max(axis=1)
combined_df["FUND_MOD4"] = combined_df[["FUND_MOD4_V1", "FUND_MOD4_V2"]].max(axis=1)
combined_df["FUND_MOD5"] = combined_df[["FUND_MOD5_V1", "FUND_MOD5_V2"]].max(axis=1)

In [27]:
# Drop the old columns after creating new columns
combined_df.drop(columns=["FUND_MOD1_V1", "FUND_MOD1_V2", "FUND_MOD2_V1", "FUND_MOD2_V2", 
                          "FUND_MOD3_V1", "FUND_MOD3_V2", "FUND_MOD4_V1", "FUND_MOD4_V2", 
                          "FUND_MOD5_V1", "FUND_MOD5_V2"], inplace=True)

In [28]:
combined_df.columns

Index(['ID', 'NFTREP_V1', 'SPEAKER_ETHCC_PARIS23', 'STUDY_GROUP_HOST_C2_22_23',
       'ETHCC_23', 'FUND_AUTHOR', 'STUDY_GROUP_HOST_360_22',
       'STUDY_GROUP_HOST_FUND_22_23', 'SPEAKER_BARCAMP_PARIS_23',
       'BARCAMP_PARIS_23', 'TEAM_BARCAMP_PARIS_23', 'FUND_MOD1', 'FUND_MOD2',
       'FUND_MOD3', 'FUND_MOD4', 'FUND_MOD5'],
      dtype='object')

In [29]:
combined_df.head()

,ID,NFTREP_V1,SPEAKER_ETHCC_PARIS23,STUDY_GROUP_HOST_C2_22_23,ETHCC_23,FUND_AUTHOR,STUDY_GROUP_HOST_360_22,STUDY_GROUP_HOST_FUND_22_23,SPEAKER_BARCAMP_PARIS_23,BARCAMP_PARIS_23,TEAM_BARCAMP_PARIS_23,FUND_MOD1,FUND_MOD2,FUND_MOD3,FUND_MOD4,FUND_MOD5
0,0xbb8743ea733155fe5e81ed285aea72cc19b2ca87,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0x049debbad61a20e21e872b06ef4f25be1253c802,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1
2,0xa481db1ac55683dfb2847c02e417b7e411bcbbea,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1
3,0x024bd3d1c3b6ba2277a744d7a99fbade9404a370,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1


In [30]:
combined_df.columns

Index(['ID', 'NFTREP_V1', 'SPEAKER_ETHCC_PARIS23', 'STUDY_GROUP_HOST_C2_22_23',
       'ETHCC_23', 'FUND_AUTHOR', 'STUDY_GROUP_HOST_360_22',
       'STUDY_GROUP_HOST_FUND_22_23', 'SPEAKER_BARCAMP_PARIS_23',
       'BARCAMP_PARIS_23', 'TEAM_BARCAMP_PARIS_23', 'FUND_MOD1', 'FUND_MOD2',
       'FUND_MOD3', 'FUND_MOD4', 'FUND_MOD5'],
      dtype='object')

In [37]:
for col in combined_df.columns:
    print(f"{col}: int")

ID: int
NFTREP_V1: int
SPEAKER_ETHCC_PARIS23: int
STUDY_GROUP_HOST_C2_22_23: int
ETHCC_23: int
FUND_AUTHOR: int
STUDY_GROUP_HOST_360_22: int
STUDY_GROUP_HOST_FUND_22_23: int
SPEAKER_BARCAMP_PARIS_23: int
BARCAMP_PARIS_23: int
TEAM_BARCAMP_PARIS_23: int
FUND_MOD1: int
FUND_MOD2: int
FUND_MOD3: int
FUND_MOD4: int
FUND_MOD5: int


In [38]:
type(combined_df['ID'].iloc[0])

str

In [31]:
print(f"There are {len(combined_df.columns)} total columns.")

There are 16 total columns.


In [32]:
print(f" There are {combined_df.duplicated().sum()} duplicated rows.")

 There are 0 duplicated rows.


In [33]:
print(f"There are {combined_df.isna().sum().sum()} missing entries.")

There are 0 missing entries.


Finally, we save this data for ease of use. 

In [34]:
final_filename = "nft_data_may_22_cleaned.csv"
combined_df.to_csv(final_filename)